In [ ]:
USE Portfolio;
SELECT * FROM dbo.ndx_index_data;
SELECT * FROM cadusd_forex_data;
GO

--helpful table for calculations
SELECT * INTO portfolio_OC_data
FROM(
	SELECT	CAST(((stock.NASDAQ_100_O_Price + stock.NASDAQ_100_C_Price)/2) 
				AS DECIMAL(18,6)) AS NASDAQ_100_OC_AVG,
			CAST(((stock.IJH_O_Price + stock.IJH_C_Price)/2) 
				AS DECIMAL(18,6)) AS IJH_OC_AVG,
			CAST(((stock.AMD_O_Price + stock.AMD_C_Price)/2) 
				AS DECIMAL(18,6)) AS AMD_OC_AVG,
			CAST(((stock.CL_O_Price + stock.CL_C_Price)/2) 
				AS DECIMAL(18,6)) AS CL_OC_AVG,
			CAST(((stock.HSY_O_Price + stock.HSY_C_Price)/2) 
				AS DECIMAL(18,6)) AS HSY_OC_AVG,
			CAST(((stock.PG_O_Price + stock.PG_C_Price)/2) 
				AS DECIMAL(18,6)) AS PG_OC_AVG,
			CAST(((stock.USPH_O_Price + stock.USPH_C_Price)/2) 
				AS DECIMAL(18,6)) AS USPH_OC_AVG,
			CAST(((cadusd.Open_Price + cadusd.Close_Price)/2) 
				AS DECIMAL(18,6)) AS CADUSD_OC_AVG,
			Start_DT =  CASE 
							WHEN stock.Start_DT IS NULL THEN cadusd.Start_time
							ELSE stock.Start_DT
						END
		FROM cadusd_forex_data cadusd FULL JOIN(
		SELECT n.Open_Price AS NASDAQ_100_O_Price,
			   n.Close_Price AS NASDAQ_100_C_Price,
			   investments_OC_prices.IJH_O_Price,
			   investments_OC_prices.IJH_C_Price,
			   investments_OC_prices.AMD_O_Price,
			   investments_OC_prices.AMD_C_Price,
			   investments_OC_prices.CL_O_Price,
			   investments_OC_prices.CL_C_Price,
			   investments_OC_prices.HSY_O_Price,
			   investments_OC_prices.HSY_C_Price,
			   investments_OC_prices.PG_O_Price,
			   investments_OC_prices.PG_C_Price,
			   investments_OC_prices.USPH_O_Price,
			   investments_OC_prices.USPH_C_Price,
			   Start_DT = CASE
							WHEN investments_OC_prices.Start_Time IS NULL THEN n.Start_Time
							ELSE investments_OC_prices.Start_Time
							END
			FROM ndx_index_data n FULL JOIN(
			SELECT uiphc.*,
				   a.Open_Price AS AMD_O_Price,
				   a.Close_Price AS AMD_C_Price
				FROM amd_stock_data a FULL JOIN 
				(SELECT uiph.*,
						c.Open_Price AS CL_O_Price,
						c.Close_Price AS CL_C_Price
					FROM cl_stock_data c FULL JOIN 
					(SELECT uip.*,
							h.Open_Price AS HSY_O_Price,
							h.Close_Price AS HSY_C_Price
						FROM hsy_stock_data h FULL JOIN
						(SELECT ui.*, 
								p.Open_Price AS PG_O_Price,
								p.Close_Price AS PG_C_Price
							FROM pg_stock_data p FULL JOIN
							(SELECT u.Open_Price AS USPH_O_Price,
									u.Close_Price AS USPH_C_Price,
									i.Open_Price AS IJH_O_Price,
									i.Close_Price AS IJH_C_Price,
									u.Start_Time
								FROM usph_stock_data u FULL JOIN ijh_etf_data i
								ON FORMAT(u.Start_Time, 'yyyy-MM-dd') = FORMAT(i.Start_Time, 'yyyy-MM-dd')
							) AS ui
							ON FORMAT(ui.Start_Time, 'yyyy-MM-dd') = FORMAT(p.Start_Time, 'yyyy-MM-dd')
						) AS uip
						ON FORMAT(uip.Start_Time, 'yyyy-MM-dd') = FORMAT(h.Start_Time, 'yyyy-MM-dd')
					) AS uiph
					ON FORMAT(uiph.Start_Time, 'yyyy-MM-dd') = FORMAT(c.Start_Time, 'yyyy-MM-dd')
				) AS uiphc
				ON FORMAT(uiphc.Start_Time, 'yyyy-MM-dd') = FORMAT(a.Start_Time, 'yyyy-MM-dd')
			) AS investments_OC_prices
			ON FORMAT(investments_OC_prices.Start_Time, 'yyyy-MM-dd') = FORMAT(n.Start_Time, 'yyyy-MM-dd')
		) AS stock
		ON FORMAT(cadusd.Start_Time, 'yyyy-MM-dd') = FORMAT(stock.Start_DT, 'yyyy-MM-dd')
)AS oc_investments;

--Create oc avg tables for each enitity
SELECT CONVERT(DATETIME, DT_ID) DT_ID, NDX_OC_AVG INTO ndx_oc_avg 
FROM(SELECT
	    FORMAT(Start_Time, 'yyyy-MM-dd') AS DT_ID,
		CAST(((ndx.Open_Price + ndx.Close_Price)/2) 
					AS DECIMAL(18,6)) AS NDX_OC_AVG
	FROM ndx_index_data ndx) AS ndx_oc;

SELECT CONVERT(DATETIME, DT_ID) DT_ID, AMD_OC_AVG INTO amd_oc_avg 
FROM(SELECT
	    FORMAT(Start_Time, 'yyyy-MM-dd') AS DT_ID,
		CAST(((amd.Open_Price + amd.Close_Price)/2) 
					AS DECIMAL(18,6)) AS AMD_OC_AVG
	FROM amd_stock_data amd) AS amd_oc;

SELECT CONVERT(DATETIME, DT_ID) DT_ID, CL_OC_AVG INTO cl_oc_avg 
FROM(SELECT
	    FORMAT(Start_Time, 'yyyy-MM-dd') AS DT_ID,
		CAST(((cl.Open_Price + cl.Close_Price)/2) 
					AS DECIMAL(18,6)) AS CL_OC_AVG
	FROM cl_stock_data cl) AS cl_oc;

SELECT CONVERT(DATETIME, DT_ID) DT_ID, HSY_OC_AVG INTO hsy_oc_avg 
FROM(SELECT
	    FORMAT(Start_Time, 'yyyy-MM-dd') AS DT_ID,
		CAST(((hsy.Open_Price + hsy.Close_Price)/2) 
					AS DECIMAL(18,6)) AS HSY_OC_AVG
	FROM hsy_stock_data hsy) AS hsy_oc;

SELECT CONVERT(DATETIME, DT_ID) DT_ID, IJH_OC_AVG INTO ijh_oc_avg 
FROM(SELECT
	    FORMAT(Start_Time, 'yyyy-MM-dd') AS DT_ID,
		CAST(((ijh.Open_Price + ijh.Close_Price)/2) 
					AS DECIMAL(18,6)) AS IJH_OC_AVG
	FROM ijh_etf_data ijh) AS ijh_oc;

SELECT CONVERT(DATETIME, DT_ID) DT_ID, PG_OC_AVG INTO pg_oc_avg 
FROM(SELECT
	    FORMAT(Start_Time, 'yyyy-MM-dd') AS DT_ID,
		CAST(((pg.Open_Price + pg.Close_Price)/2) 
					AS DECIMAL(18,6)) AS PG_OC_AVG
	FROM pg_stock_data pg) AS pg_oc;

SELECT CONVERT(DATETIME, DT_ID) DT_ID, USPH_OC_AVG INTO usph_oc_avg 
FROM(SELECT
	    FORMAT(Start_Time, 'yyyy-MM-dd') AS DT_ID,
		CAST(((usph.Open_Price + usph.Close_Price)/2) 
					AS DECIMAL(18,6)) AS USPH_OC_AVG
	FROM usph_stock_data usph) AS usph_oc;

SELECT CONVERT(DATETIME, DT_ID) DT_ID, CADUSD_OC_AVG INTO cadusd_oc_avg 
FROM(SELECT
	    FORMAT(Start_Time, 'yyyy-MM-dd') AS DT_ID,
		CAST(((cadusd.Open_Price + cadusd.Close_Price)/2) 
					AS DECIMAL(18,6)) AS CADUSD_OC_AVG
	FROM cadusd_forex_data cadusd) AS cadusd_oc;

SELECT * FROM cadusd_oc_avg;
SELECT * FROM amd_oc_avg;
SELECT * FROM cl_oc_avg;
SELECT * FROM pg_oc_avg;
SELECT * FROM usph_oc_avg;
SELECT * FROM hsy_oc_avg;
SELECT * FROM ijh_oc_avg;
SELECT * FROM ndx_oc_avg;

SELECT * FROM portfolio_OC_data;


--Get the first average OC price insert into ticker table
DECLARE @ndx_start DECIMAL(18,6);
SET @ndx_start = CAST((SELECT TOP 1 NDX_OC_AVG FROM ndx_oc_avg) AS DECIMAL(18,6));


DECLARE @buy_price_amd DECIMAL(18,6);
SET @buy_price_amd = CAST((SELECT TOP 1 AMD_OC_AVG FROM amd_oc_avg) AS DECIMAL(18,6));

DECLARE @buy_price_cadusd DECIMAL(18,6);
SET @buy_price_cadusd = CAST((SELECT TOP 1 CADUSD_OC_AVG FROM cadusd_oc_avg) AS DECIMAL(18,6));

DECLARE @buy_price_cl DECIMAL(18,6);
SET @buy_price_cl = CAST((SELECT TOP 1 CL_OC_AVG FROM cl_oc_avg) AS DECIMAL(18,6));

DECLARE @buy_price_hsy DECIMAL(18,6);
SET @buy_price_hsy = CAST((SELECT TOP 1 HSY_OC_AVG FROM hsy_oc_avg) AS DECIMAL(18,6));

DECLARE @buy_price_ijh DECIMAL(18,6);
SET @buy_price_ijh = CAST((SELECT TOP 1 IJH_OC_AVG FROM ijh_oc_avg) AS DECIMAL(18,6));

DECLARE @buy_price_pg DECIMAL(18,6);
SET @buy_price_pg = CAST((SELECT TOP 1 PG_OC_AVG FROM pg_oc_avg) AS DECIMAL(18,6));

DECLARE @buy_price_usph DECIMAL(18,6);
SET @buy_price_usph = CAST((SELECT TOP 1 USPH_OC_AVG FROM usph_oc_avg) AS DECIMAL(18,6));

--Create tick table
CREATE TABLE tick_table(
	ticker VARCHAR(6) PRIMARY KEY,
	buy_in_price DECIMAL(18,6),
	full_name VARCHAR(60),
	number_purchased INTEGER,
	portf_weight FLOAT,
	div_value DECIMAL(5,4),
	div_date DATETIME
);

--fill in the table
DECLARE @portf_value DECIMAL(18,6);
SET @portf_value = 100000.0-0.378015414
INSERT tick_table(ticker, buy_in_price, full_name, number_purchased, portf_weight, div_value, div_date)
 VALUES('AMD', @buy_price_amd, 'AMD', 219, ((219*@buy_price_amd)/@portf_value), 0, NULL),
		('CL', @buy_price_cl, 'Colagate-Palmolive Company', 180, ((180*@buy_price_cl)/@portf_value), .0206, DATEFROMPARTS(2023,7,19)),
		('HSY', @buy_price_hsy, 'The Hershy Company', 62, ((62*@buy_price_hsy)/@portf_value), .0298, DATEFROMPARTS(2023,6,16)),
		('IJH', @buy_price_ijh, 'iShares Core S&P Mid-Cap ETF', 301, ((301*@buy_price_ijh)/@portf_value), 0, NULL),
		('PG', @buy_price_pg, 'The Procter & Gamble Company', 94, ((94*@buy_price_pg)/@portf_value), .0244, DATEFROMPARTS(2023,4,18)),
		('USPH', @buy_price_usph, 'U.S. Physical Therapy, Inc.', 175, ((175*@buy_price_usph)/@portf_value), .0191, DATEFROMPARTS(2023,6,22)),
		('CADUSD', @buy_price_cadusd, 'CAD/USD Forex', 19408, ((19408*@buy_price_cadusd)/@portf_value), 0, NULL),
		('NDX', @ndx_start, 'NASDAQ 100', 0, 0, 0, NULL);
GO

SELECT * FROM tick_table;

SELECT * FROM ijh_oc_avg;
SELECT * FROM ndx_oc_avg;

--This will be our fact table
CREATE TABLE portf_table(
	date_ID DATETIME,
	ticker_ID VARCHAR(6),
	day_10_avg DECIMAL(18,6),
	day_100_avg DECIMAL(18,6),
	total_return DECIMAL(18,6),
	cumulative_return DECIMAL(18,6),
	annualized_return DECIMAL(18,6),
	volatility FLOAT,
	sharpe_ratio FLOAT,
	beta FLOAT,
	percentage_change FLOAT,
	PRIMARY KEY(date_id, ticker_id)
);
		
SELECT * FROM portf_table;

--Insert agg data from market index
DECLARE @ndx_buy DECIMAL(18,6);
SET @ndx_buy = (SELECT buy_in_price FROM tick_table WHERE ticker = 'NDX');
WITH count_ndx AS (
	SELECT *, 
		LAG(NDX_OC_AVG) OVER(ORDER BY DT_ID) AS prev_price,
		COUNT(*) OVER(ORDER BY DT_ID ROWS BETWEEN 
			UNBOUNDED PRECEDING AND CURRENT ROW) AS row_num
	FROM ndx_oc_avg
),
daily_log AS(
	SELECT *,
		LOG(NDX_OC_AVG/prev_price) AS log_d
	FROM count_ndx
),
ndx_daily_volatility AS(
	SELECT *,
		100*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
		BETWEEN 252 PRECEDING AND CURRENT ROW) AS daily_vaolatility
	FROM daily_log
),
ndx_historic_vol AS(
	SELECT *,
	100*SQRT(252)*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
	BETWEEN 252 PRECEDING AND CURRENT ROW) AS historic_vaolatility
	FROM ndx_daily_volatility
),
ndx_total_returns AS(--same as cumlative returns
	SELECT *,
		(NDX_OC_AVG-@ndx_buy) AS total_return
	FROM ndx_historic_vol
),
ndx_cummulative_returns AS(
	SELECT *,
		(NDX_OC_AVG-@ndx_buy)/(@ndx_buy) as cumm_return
	FROM ndx_total_returns
),
ndx_annualized_returns AS(--!!!!Change 252 to 365 for other non-stock/etf calculations
	SELECT *,
		annual_return = CASE
							WHEN cumm_return < -1.0 THEN 
									-1.0*POWER(ABS(cumm_return+1), 252.0/row_num)-1
							ELSE POWER(cumm_return+1, 252.0/row_num)-1
						END
	FROM ndx_cummulative_returns
)
INSERT INTO portf_table(date_ID, ticker_ID, day_10_avg, day_100_avg, volatility, percentage_change,
						total_return, cumulative_return, annualized_return)
SELECT date_ID = DT_ID,
		ticker_ID = 'NDX',
		day_10_avg = AVG(NDX_OC_AVG) OVER(ORDER BY DT_ID ASC
							ROWS BETWEEN 9 PRECEDING AND CURRENT ROW),
		day_100_avg = AVG(NDX_OC_AVG) OVER(ORDER BY DT_ID ASC
							ROWS BETWEEN 99 PRECEDING AND CURRENT ROW),
		volatility = historic_vaolatility,
		percentage_change = 100*((NDX_OC_AVG-prev_price)/prev_price),
		total_return = total_return,
		cumulative_return = 100*cumm_return,
		annualized_return = 100*annual_return
FROM ndx_annualized_returns
GO

--Put all our individual investment aggs into our fact table
DECLARE @amd_buy DECIMAL(18,6);
SET @amd_buy = (SELECT buy_in_price FROM tick_table WHERE ticker = 'AMD');
DECLARE @amd_shares INTEGER;
SET @amd_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'AMD');
WITH count_amd AS (
	SELECT *, 
		LAG(AMD_OC_AVG) OVER(ORDER BY DT_ID) AS prev_price,
		COUNT(*) OVER(ORDER BY DT_ID ROWS BETWEEN 
			UNBOUNDED PRECEDING AND CURRENT ROW) AS row_num
	FROM amd_oc_avg
),
daily_log AS(--logarithmic returns
	SELECT *,
		LOG(AMD_OC_AVG/prev_price) AS log_d
	FROM count_amd
),
amd_daily_volatility AS(
	SELECT *,
		100*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
		BETWEEN 252 PRECEDING AND CURRENT ROW) AS daily_vaolatility
	FROM daily_log
),
amd_historic_vol AS(
	SELECT *,
	100*SQRT(252)*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
	BETWEEN 252 PRECEDING AND CURRENT ROW) AS historic_vaolatility
	FROM amd_daily_volatility
),
amd_total_returns AS(
	SELECT *,
		@amd_shares*(AMD_OC_AVG-@amd_buy) AS total_return
	FROM amd_historic_vol
),
amd_cummulative_returns AS(
	SELECT *,
		(AMD_OC_AVG-@amd_buy)/(@amd_buy) as cumm_return
	FROM amd_total_returns
),
amd_annualized_returns AS(--!!!!Change 252 to 365 for other non-stock/etf calculations
	SELECT *,
		annual_return = CASE
							WHEN cumm_return < -1.0 THEN 
									-1.0*POWER(ABS(cumm_return+1), 252.0/row_num)-1
							ELSE POWER(cumm_return+1, 252.0/row_num)-1
						END
	FROM amd_cummulative_returns
),
amd_daily_returns AS(
	SELECT acr.*,
		unified_date = CASE 
							WHEN market_idx.date_ID IS NULL THEN acr.DT_ID
							ELSE market_idx.date_ID
						END,
		100*((AMD_OC_AVG-prev_price)/(prev_price)) AS amd_daily_ret,
		market_idx.percentage_change AS mrkt_dailt_ret
		FROM amd_annualized_returns acr FULL JOIN (
		SELECT percentage_change, date_ID FROM portf_table
		WHERE ticker_ID = 'NDX') AS market_idx
		ON acr.DT_ID = market_idx.date_ID
),
amd_avg_daily_ret AS(
	SELECT *,
		AVG(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 252 PRECEDING AND CURRENT ROW) AS idx_avg_day_ret,
		AVG(amd_daily_ret) OVER(ORDER BY unified_date ROWS BETWEEN 252
				PRECEDING AND CURRENT ROW) AS amd_avg_day_ret
		FROM amd_daily_returns
),
amd_covar_sums AS(
		SELECT *,
			SUM(((amd_daily_ret)-(amd_avg_day_ret))*((mrkt_dailt_ret)-(idx_avg_day_ret)))
			OVER(ORDER BY unified_date ROWS BETWEEN 252 PRECEDING AND CURRENT ROW)--convert annual return from decimal to percent
			AS covar_sum
		FROM amd_avg_daily_ret
),
amd_variances AS(
		SELECT *,
			VAR(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 252 PRECEDING AND CURRENT ROW) AS var_idx,--252 or unbounded?
			covar_sum/(row_num-1) AS covar_stocks
		FROM amd_covar_sums
),
amd_beta AS(
		SELECT *,
		(covar_stocks/var_idx) AS amd_beta
		FROM amd_variances

)
INSERT INTO portf_table(date_ID, ticker_ID, day_10_avg, day_100_avg, total_return, cumulative_return,
						annualized_return, volatility, beta, percentage_change)
SELECT date_ID = DT_ID,
	ticker_ID = 'AMD',
	day_10_avg = AVG(AMD_OC_AVG) OVER(ORDER BY DT_ID ASC
							ROWS BETWEEN 9 PRECEDING AND CURRENT ROW),
	day_100_avg = AVG(AMD_OC_AVG) OVER(ORDER BY DT_ID ASC
						ROWS BETWEEN 99 PRECEDING AND CURRENT ROW),
	total_return = amd_beta.total_return,
	cumulative_return = 100*cumm_return,
	annualized_return = 100*amd_beta.annual_return,
	volatility = amd_beta.historic_vaolatility,
	beta =  amd_beta.amd_beta,
	percentage_change = 100*((AMD_OC_AVG-prev_price)/prev_price)
FROM amd_beta
WHERE DT_ID IS NOT NULL;
GO


DECLARE @ijh_buy DECIMAL(18,6);
SET @ijh_buy = (SELECT buy_in_price FROM tick_table WHERE ticker = 'IJH');
DECLARE @ijh_shares INTEGER;
SET @ijh_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'IJH');
WITH count_ijh AS (
	SELECT *, 
		LAG(IJH_OC_AVG) OVER(ORDER BY DT_ID) AS prev_price,
		COUNT(*) OVER(ORDER BY DT_ID ROWS BETWEEN 
			UNBOUNDED PRECEDING AND CURRENT ROW) AS row_num
	FROM ijh_oc_avg
),
daily_log AS(
	SELECT *,
		LOG(IJH_OC_AVG/prev_price) AS log_d
	FROM count_ijh
),
ijh_daily_volatility AS(
	SELECT *,
		100*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
		BETWEEN 252 PRECEDING AND CURRENT ROW) AS daily_vaolatility
	FROM daily_log
),
ijh_historic_vol AS(
	SELECT *,
	100*SQRT(252)*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
	BETWEEN 252 PRECEDING AND CURRENT ROW) AS historic_vaolatility
	FROM ijh_daily_volatility
),
ijh_total_returns AS(
	SELECT *,
		@ijh_shares*(IJH_OC_AVG-@ijh_buy) AS total_return
	FROM ijh_historic_vol
),
ijh_cummulative_returns AS(
	SELECT *,
		(IJH_OC_AVG-@ijh_buy)/(@ijh_buy) as cumm_return
	FROM ijh_total_returns
),
ijh_annualized_returns AS(--!!!!Change 252 to 365 for other non-stock/etf calculations
	SELECT *,
		annual_return = CASE
							WHEN cumm_return < -1.0 THEN 
									-1.0*POWER(ABS(cumm_return+1), 252.0/row_num)-1
							ELSE POWER(cumm_return+1, 252.0/row_num)-1
						END
	FROM ijh_cummulative_returns
),
ijh_daily_returns AS(
	SELECT acr.*,
		unified_date = CASE 
							WHEN market_idx.date_ID IS NULL THEN acr.DT_ID
							ELSE market_idx.date_ID
						END,
		100*((IJH_OC_AVG-prev_price)/(prev_price)) AS ijh_daily_ret,
		market_idx.percentage_change AS mrkt_dailt_ret
		FROM ijh_annualized_returns acr FULL JOIN (
		SELECT percentage_change, date_ID FROM portf_table
		WHERE ticker_ID = 'NDX') AS market_idx
		ON acr.DT_ID = market_idx.date_ID
),
ijh_avg_daily_ret AS(
	SELECT *,
		AVG(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 252 PRECEDING AND CURRENT ROW) AS idx_avg_day_ret,
		AVG(ijh_daily_ret) OVER(ORDER BY unified_date ROWS BETWEEN 252
				PRECEDING AND CURRENT ROW) AS ijh_avg_day_ret
		FROM ijh_daily_returns
),
ijh_covar_sums AS(
		SELECT *,
			SUM(((ijh_daily_ret)-(ijh_avg_day_ret))*((mrkt_dailt_ret)-(idx_avg_day_ret)))
			OVER(ORDER BY unified_date ROWS BETWEEN 252 PRECEDING AND CURRENT ROW)--convert annual return from decimal to percent
			AS covar_sum
		FROM ijh_avg_daily_ret
),
ijh_variances AS(
		SELECT *,
			VAR(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 252 PRECEDING AND CURRENT ROW) AS var_idx,--252 or unbounded?
			covar_sum/(row_num-1) AS covar_stocks
		FROM ijh_covar_sums
),
ijh_beta AS(
		SELECT *,
		(covar_stocks/var_idx) AS ijh_beta
		FROM ijh_variances

)
INSERT INTO portf_table(date_ID, ticker_ID, day_10_avg, day_100_avg, total_return, cumulative_return,
annualized_return, volatility, beta, percentage_change)
SELECT date_ID = DT_ID,
	ticker_ID = 'IJH',
	day_10_avg = AVG(IJH_OC_AVG) OVER(ORDER BY DT_ID ASC
							ROWS BETWEEN 9 PRECEDING AND CURRENT ROW),
	day_100_avg = AVG(IJH_OC_AVG) OVER(ORDER BY DT_ID ASC
						ROWS BETWEEN 99 PRECEDING AND CURRENT ROW),
	total_return = ijh_beta.total_return,
	cumulative_return = 100*ijh_beta.cumm_return,
	annualized_return = 100*ijh_beta.annual_return,
	volatility = ijh_beta.historic_vaolatility,
	beta =  ijh_beta.ijh_beta,
	percentage_change = 100*((IJH_OC_AVG-prev_price)/prev_price)
FROM ijh_beta
WHERE DT_ID IS NOT NULL;
GO




DECLARE @hsy_buy DECIMAL(18,6);
SET @hsy_buy = (SELECT buy_in_price FROM tick_table WHERE ticker = 'HSY');
DECLARE @hsy_shares INTEGER;
SET @hsy_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'HSY');
DECLARE @hsy_div DECIMAL(5,4);
SET @hsy_div = (SELECT div_value FROM tick_table WHERE ticker = 'HSY');
DECLARE @hsy_div_date DATETIME;
SET @hsy_div_date = (SELECT div_date FROM tick_table WHERE ticker = 'HSY');
WITH count_hsy AS (
	SELECT *, 
		LAG(HSY_OC_AVG) OVER(ORDER BY DT_ID) AS prev_price,
		COUNT(*) OVER(ORDER BY DT_ID ROWS BETWEEN 
			UNBOUNDED PRECEDING AND CURRENT ROW) AS row_num
	FROM hsy_oc_avg
),
daily_log AS(
	SELECT *,
		LOG(HSY_OC_AVG/prev_price) AS log_d
	FROM count_hsy
),
hsy_daily_volatility AS(
	SELECT *,
		100*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
		BETWEEN 252 PRECEDING AND CURRENT ROW) AS daily_vaolatility
	FROM daily_log
),
hsy_historic_vol AS(
	SELECT *,
	100*SQRT(252)*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
	BETWEEN 252 PRECEDING AND CURRENT ROW) AS historic_vaolatility
	FROM hsy_daily_volatility
),
hsy_div_pay AS(
	SELECT *,
	div_pay = CASE 
					WHEN @hsy_div_date = DT_ID THEN
							@hsy_div*@hsy_shares*HSY_OC_AVG
					ELSE 0.0
				END
	FROM hsy_historic_vol
),
hsy_lifetime_divpay AS(
	SELECT *,
	SUM(div_pay) OVER(ORDER BY DT_ID ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS div_life_pay
	FROM hsy_div_pay
),
hsy_total_returns AS(
	SELECT *,
		(@hsy_shares*(HSY_OC_AVG-@hsy_buy))+div_life_pay AS total_return
	FROM hsy_lifetime_divpay
),
hsy_cummulative_returns AS(
	SELECT *,
		(total_return)/(@hsy_buy*@hsy_shares) as cumm_return
	FROM hsy_total_returns
),
hsy_annualized_returns AS(--!!!!Change 252 to 365 for other non-stock/etf calculations
	SELECT *,
		annual_return = CASE
							WHEN cumm_return < -1.0 THEN 
									-1.0*POWER(ABS(cumm_return+1), 252.0/row_num)-1
							ELSE POWER(cumm_return+1, 252.0/row_num)-1
						END
	FROM hsy_cummulative_returns
),
hsy_daily_returns AS(
	SELECT acr.*,
		unified_date = CASE 
							WHEN market_idx.date_ID IS NULL THEN acr.DT_ID
							ELSE market_idx.date_ID
						END,
		100*((HSY_OC_AVG-prev_price)/(prev_price)) AS hsy_daily_ret,
		market_idx.percentage_change AS mrkt_dailt_ret
		FROM hsy_annualized_returns acr FULL JOIN (
		SELECT percentage_change, date_ID FROM portf_table
		WHERE ticker_ID = 'NDX') AS market_idx
		ON acr.DT_ID = market_idx.date_ID
),
hsy_avg_daily_ret AS(
	SELECT *,
		AVG(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 252 PRECEDING AND CURRENT ROW) AS idx_avg_day_ret,
		AVG(hsy_daily_ret) OVER(ORDER BY unified_date ROWS BETWEEN 252
				PRECEDING AND CURRENT ROW) AS hsy_avg_day_ret
		FROM hsy_daily_returns
),
hsy_covar_sums AS(
		SELECT *,
			SUM(((hsy_daily_ret)-(hsy_avg_day_ret))*((mrkt_dailt_ret)-(idx_avg_day_ret)))
			OVER(ORDER BY unified_date ROWS BETWEEN 252 PRECEDING AND CURRENT ROW)--convert annual return from decimal to percent
			AS covar_sum
		FROM hsy_avg_daily_ret
),
hsy_variances AS(
		SELECT *,
			VAR(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 252 PRECEDING AND CURRENT ROW) AS var_idx,--252 or unbounded?
			covar_sum/(row_num-1) AS covar_stocks
		FROM hsy_covar_sums
),
hsy_beta AS(
		SELECT *,
		(covar_stocks/var_idx) AS hsy_beta
		FROM hsy_variances

)
INSERT INTO portf_table(date_ID, ticker_ID, day_10_avg, day_100_avg, total_return, cumulative_return,
annualized_return, volatility, beta, percentage_change)
SELECT date_ID = DT_ID,
	ticker_ID = 'HSY',
	day_10_avg = AVG(HSY_OC_AVG) OVER(ORDER BY DT_ID ASC
							ROWS BETWEEN 9 PRECEDING AND CURRENT ROW),
	day_100_avg = AVG(HSY_OC_AVG) OVER(ORDER BY DT_ID ASC
						ROWS BETWEEN 99 PRECEDING AND CURRENT ROW),
	total_return = hsy_beta.total_return,
	cumulative_return = 100*hsy_beta.cumm_return,
	annualized_return = 100*hsy_beta.annual_return,
	volatility = hsy_beta.historic_vaolatility,
	beta =  hsy_beta.hsy_beta,
	percentage_change = 100*((HSY_OC_AVG-prev_price)/prev_price)
FROM hsy_beta
WHERE DT_ID IS NOT NULL;
GO

SELECT * FROM tick_table;

DECLARE @cl_buy DECIMAL(18,6);
SET @cl_buy = (SELECT buy_in_price FROM tick_table WHERE ticker = 'CL');
DECLARE @cl_shares INTEGER;
SET @cl_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'CL');
DECLARE @cl_div DECIMAL(5,4);
SET @cl_div = (SELECT div_value FROM tick_table WHERE ticker = 'CL');
DECLARE @cl_div_date DATETIME;
SET @cl_div_date = (SELECT div_date FROM tick_table WHERE ticker = 'CL');
WITH count_cl AS (
	SELECT *, 
		LAG(CL_OC_AVG) OVER(ORDER BY DT_ID) AS prev_price,
		COUNT(*) OVER(ORDER BY DT_ID ROWS BETWEEN 
			UNBOUNDED PRECEDING AND CURRENT ROW) AS row_num
	FROM cl_oc_avg
),
daily_log AS(
	SELECT *,
		LOG(CL_OC_AVG/prev_price) AS log_d
	FROM count_cl
),
cl_daily_volatility AS(
	SELECT *,
		100*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
		BETWEEN 252 PRECEDING AND CURRENT ROW) AS daily_vaolatility
	FROM daily_log
),
cl_historic_vol AS(
	SELECT *,
	100*SQRT(252)*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
	BETWEEN 252 PRECEDING AND CURRENT ROW) AS historic_vaolatility
	FROM cl_daily_volatility
),
cl_div_pay AS(
	SELECT *,
	div_pay = CASE 
					WHEN @cl_div_date = DT_ID THEN
							@cl_div*@cl_shares*CL_OC_AVG
					ELSE 0.0
				END
	FROM cl_historic_vol
),
cl_lifetime_divpay AS(
	SELECT *,
	SUM(div_pay) OVER(ORDER BY DT_ID ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS div_life_pay
	FROM cl_div_pay
),
cl_total_returns AS(
	SELECT *,
		(@cl_shares*(CL_OC_AVG-@cl_buy))+div_life_pay AS total_return
	FROM cl_lifetime_divpay
),
cl_cummulative_returns AS(
	SELECT *,
		(total_return)/(@cl_buy*@cl_shares) as cumm_return
	FROM cl_total_returns
),
cl_annualized_returns AS(--!!!!Change 252 to 365 for other non-stock/etf calculations
	SELECT *,
		annual_return = CASE
							WHEN cumm_return < -1.0 THEN 
									-1.0*POWER(ABS(cumm_return+1), 252.0/row_num)-1
							ELSE POWER(cumm_return+1, 252.0/row_num)-1
						END
	FROM cl_cummulative_returns
),
cl_daily_returns AS(
	SELECT acr.*,
		unified_date = CASE 
							WHEN market_idx.date_ID IS NULL THEN acr.DT_ID
							ELSE market_idx.date_ID
						END,
		100*((CL_OC_AVG-prev_price)/(prev_price)) AS cl_daily_ret,
		market_idx.percentage_change AS mrkt_dailt_ret
		FROM cl_annualized_returns acr FULL JOIN (
		SELECT percentage_change, date_ID FROM portf_table
		WHERE ticker_ID = 'NDX') AS market_idx
		ON acr.DT_ID = market_idx.date_ID
),
cl_avg_daily_ret AS(
	SELECT *,
		AVG(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 252 PRECEDING AND CURRENT ROW) AS idx_avg_day_ret,
		AVG(cl_daily_ret) OVER(ORDER BY unified_date ROWS BETWEEN 252
				PRECEDING AND CURRENT ROW) AS cl_avg_day_ret
		FROM cl_daily_returns
),
cl_covar_sums AS(
		SELECT *,
			SUM(((cl_daily_ret)-(cl_avg_day_ret))*((mrkt_dailt_ret)-(idx_avg_day_ret)))
			OVER(ORDER BY unified_date ROWS BETWEEN 252 PRECEDING AND CURRENT ROW)--convert annual return from decimal to percent
			AS covar_sum
		FROM cl_avg_daily_ret
),
cl_variances AS(
		SELECT *,
			VAR(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 252 PRECEDING AND CURRENT ROW) AS var_idx,--252 or unbounded?
			covar_sum/(row_num-1) AS covar_stocks
		FROM cl_covar_sums
),
cl_beta AS(
		SELECT *,
		(covar_stocks/var_idx) AS cl_beta
		FROM cl_variances

)
INSERT INTO portf_table(date_ID, ticker_ID, day_10_avg, day_100_avg, total_return, cumulative_return,
annualized_return, volatility, beta, percentage_change)
SELECT date_ID = DT_ID,
	ticker_ID = 'CL',
	day_10_avg = AVG(CL_OC_AVG) OVER(ORDER BY DT_ID ASC
							ROWS BETWEEN 9 PRECEDING AND CURRENT ROW),
	day_100_avg = AVG(CL_OC_AVG) OVER(ORDER BY DT_ID ASC
						ROWS BETWEEN 99 PRECEDING AND CURRENT ROW),
	total_return = cl_beta.total_return,
	cumulative_return = 100*cl_beta.cumm_return,
	annualized_return = 100*cl_beta.annual_return,
	volatility = cl_beta.historic_vaolatility,
	beta =  cl_beta.cl_beta,
	percentage_change = 100*((CL_OC_AVG-prev_price)/prev_price)
FROM cl_beta
WHERE DT_ID IS NOT NULL;
GO



DECLARE @pg_buy DECIMAL(18,6);
SET @pg_buy = (SELECT buy_in_price FROM tick_table WHERE ticker = 'PG');
DECLARE @pg_shares INTEGER;
SET @pg_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'PG');
DECLARE @pg_div DECIMAL(5,4);
SET @pg_div = (SELECT div_value FROM tick_table WHERE ticker = 'PG');
DECLARE @pg_div_date DATETIME;
SET @pg_div_date = (SELECT div_date FROM tick_table WHERE ticker = 'PG');
WITH count_pg AS (
	SELECT *, 
		LAG(PG_OC_AVG) OVER(ORDER BY DT_ID) AS prev_price,
		COUNT(*) OVER(ORDER BY DT_ID ROWS BETWEEN 
			UNBOUNDED PRECEDING AND CURRENT ROW) AS row_num
	FROM pg_oc_avg
),
daily_log AS(
	SELECT *,
		LOG(PG_OC_AVG/prev_price) AS log_d
	FROM count_pg
),
pg_daily_volatility AS(
	SELECT *,
		100*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
		BETWEEN 252 PRECEDING AND CURRENT ROW) AS daily_vaolatility
	FROM daily_log
),
pg_historic_vol AS(
	SELECT *,
	100*SQRT(252)*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
	BETWEEN 252 PRECEDING AND CURRENT ROW) AS historic_vaolatility
	FROM pg_daily_volatility
),
pg_div_pay AS(
	SELECT *,
	div_pay = CASE 
					WHEN @pg_div_date = DT_ID THEN
							@pg_div*@pg_shares*PG_OC_AVG
					ELSE 0.0
				END
	FROM pg_historic_vol
),
pg_lifetime_divpay AS(
	SELECT *,
	SUM(div_pay) OVER(ORDER BY DT_ID ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS div_life_pay
	FROM pg_div_pay
),
pg_total_returns AS(
	SELECT *,
		(@pg_shares*(PG_OC_AVG-@pg_buy))+div_life_pay AS total_return
	FROM pg_lifetime_divpay
),
pg_cummulative_returns AS(
	SELECT *,
		(total_return)/(@pg_buy*@pg_shares) as cumm_return
	FROM pg_total_returns
),
pg_annualized_returns AS(--!!!!Change 252 to 365 for other non-stock/etf calculations
	SELECT *,
		annual_return = CASE
							WHEN cumm_return < -1.0 THEN 
									-1.0*POWER(ABS(cumm_return+1), 252.0/row_num)-1
							ELSE POWER(cumm_return+1, 252.0/row_num)-1
						END
	FROM pg_cummulative_returns
),
pg_daily_returns AS(
	SELECT acr.*,
		unified_date = CASE 
							WHEN market_idx.date_ID IS NULL THEN acr.DT_ID
							ELSE market_idx.date_ID
						END,
		100*((PG_OC_AVG-prev_price)/(prev_price)) AS pg_daily_ret,
		market_idx.percentage_change AS mrkt_dailt_ret
		FROM pg_annualized_returns acr FULL JOIN (
		SELECT percentage_change, date_ID FROM portf_table
		WHERE ticker_ID = 'NDX') AS market_idx
		ON acr.DT_ID = market_idx.date_ID
),
pg_avg_daily_ret AS(
	SELECT *,
		AVG(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 252 PRECEDING AND CURRENT ROW) AS idx_avg_day_ret,
		AVG(pg_daily_ret) OVER(ORDER BY unified_date ROWS BETWEEN 252
				PRECEDING AND CURRENT ROW) AS pg_avg_day_ret
		FROM pg_daily_returns
),
pg_covar_sums AS(
		SELECT *,
			SUM(((pg_daily_ret)-(pg_avg_day_ret))*((mrkt_dailt_ret)-(idx_avg_day_ret)))
			OVER(ORDER BY unified_date ROWS BETWEEN 252 PRECEDING AND CURRENT ROW)--convert annual return from decimal to percent
			AS covar_sum
		FROM pg_avg_daily_ret
),
pg_variances AS(
		SELECT *,
			VAR(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 252 PRECEDING AND CURRENT ROW) AS var_idx,--252 or unbounded?
			covar_sum/(row_num-1) AS covar_stocks
		FROM pg_covar_sums
),
pg_beta AS(
		SELECT *,
		(covar_stocks/var_idx) AS pg_beta
		FROM pg_variances

)
INSERT INTO portf_table(date_ID, ticker_ID, day_10_avg, day_100_avg, total_return, cumulative_return,
annualized_return, volatility, beta, percentage_change)
SELECT date_ID = DT_ID,
	ticker_ID = 'PG',
	day_10_avg = AVG(PG_OC_AVG) OVER(ORDER BY DT_ID ASC
							ROWS BETWEEN 9 PRECEDING AND CURRENT ROW),
	day_100_avg = AVG(PG_OC_AVG) OVER(ORDER BY DT_ID ASC
						ROWS BETWEEN 99 PRECEDING AND CURRENT ROW),
	total_return = pg_beta.total_return,
	cumulative_return = 100*pg_beta.cumm_return,
	annualized_return = 100*pg_beta.annual_return,
	volatility = pg_beta.historic_vaolatility,
	beta =  pg_beta.pg_beta,
	percentage_change = 100*((PG_OC_AVG-prev_price)/prev_price)
FROM pg_beta
WHERE DT_ID IS NOT NULL;
GO



DECLARE @usph_buy DECIMAL(18,6);
SET @usph_buy = (SELECT buy_in_price FROM tick_table WHERE ticker = 'USPH');
DECLARE @usph_shares INTEGER;
SET @usph_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'USPH');
DECLARE @usph_div DECIMAL(5,4);
SET @usph_div = (SELECT div_value FROM tick_table WHERE ticker = 'USPH');
DECLARE @usph_div_date DATETIME;
SET @usph_div_date = (SELECT div_date FROM tick_table WHERE ticker = 'USPH');
WITH count_usph AS (
	SELECT *, 
		LAG(USPH_OC_AVG) OVER(ORDER BY DT_ID) AS prev_price,
		COUNT(*) OVER(ORDER BY DT_ID ROWS BETWEEN 
			UNBOUNDED PRECEDING AND CURRENT ROW) AS row_num
	FROM usph_oc_avg
),
daily_log AS(
	SELECT *,
		LOG(USPH_OC_AVG/prev_price) AS log_d
	FROM count_usph
),
usph_daily_volatility AS(
	SELECT *,
		100*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
		BETWEEN 252 PRECEDING AND CURRENT ROW) AS daily_vaolatility
	FROM daily_log
),
usph_historic_vol AS(
	SELECT *,
	100*SQRT(252)*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
	BETWEEN 252 PRECEDING AND CURRENT ROW) AS historic_vaolatility
	FROM usph_daily_volatility
),
usph_div_pay AS(
	SELECT *,
	div_pay = CASE 
					WHEN @usph_div_date = DT_ID THEN
							@usph_div*@usph_shares*USPH_OC_AVG
					ELSE 0.0
				END
	FROM usph_historic_vol
),
usph_lifetime_divpay AS(
	SELECT *,
	SUM(div_pay) OVER(ORDER BY DT_ID ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS div_life_pay
	FROM usph_div_pay
),
usph_total_returns AS(
	SELECT *,
		(@usph_shares*(USPH_OC_AVG-@usph_buy))+div_life_pay AS total_return
	FROM usph_lifetime_divpay
),
usph_cummulative_returns AS(
	SELECT *,
		(total_return)/(@usph_buy*@usph_shares) as cumm_return
	FROM usph_total_returns
),
usph_annualized_returns AS(--!!!!Change 252 to 365 for other non-stock/etf calculations
	SELECT *,
		annual_return = CASE
							WHEN cumm_return < -1.0 THEN 
									-1.0*POWER(ABS(cumm_return+1), 252.0/row_num)-1
							ELSE POWER(cumm_return+1, 252.0/row_num)-1
						END
	FROM usph_cummulative_returns
),
usph_daily_returns AS(
	SELECT acr.*,
		unified_date = CASE 
							WHEN market_idx.date_ID IS NULL THEN acr.DT_ID
							ELSE market_idx.date_ID
						END,
		100*((USPH_OC_AVG-prev_price)/(prev_price)) AS usph_daily_ret,
		market_idx.percentage_change AS mrkt_dailt_ret
		FROM usph_annualized_returns acr FULL JOIN (
		SELECT percentage_change, date_ID FROM portf_table
		WHERE ticker_ID = 'NDX') AS market_idx
		ON acr.DT_ID = market_idx.date_ID
),
usph_avg_daily_ret AS(
	SELECT *,
		AVG(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 252 PRECEDING AND CURRENT ROW) AS idx_avg_day_ret,
		AVG(usph_daily_ret) OVER(ORDER BY unified_date ROWS BETWEEN 252
				PRECEDING AND CURRENT ROW) AS usph_avg_day_ret
		FROM usph_daily_returns
),
usph_covar_sums AS(
		SELECT *,
			SUM(((usph_daily_ret)-(usph_avg_day_ret))*((mrkt_dailt_ret)-(idx_avg_day_ret)))
			OVER(ORDER BY unified_date ROWS BETWEEN 252 PRECEDING AND CURRENT ROW)--convert annual return from decimal to percent
			AS covar_sum
		FROM usph_avg_daily_ret
),
usph_variances AS(
		SELECT *,
			VAR(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 252 PRECEDING AND CURRENT ROW) AS var_idx,--252 or unbounded?
			covar_sum/(row_num-1) AS covar_stocks
		FROM usph_covar_sums
),
usph_beta AS(
		SELECT *,
		(covar_stocks/var_idx) AS usph_beta
		FROM usph_variances

)
INSERT INTO portf_table(date_ID, ticker_ID, day_10_avg, day_100_avg, total_return, cumulative_return,
annualized_return, volatility, beta, percentage_change)
SELECT date_ID = DT_ID,
	ticker_ID = 'USPH',
	day_10_avg = AVG(USPH_OC_AVG) OVER(ORDER BY DT_ID ASC
							ROWS BETWEEN 9 PRECEDING AND CURRENT ROW),
	day_100_avg = AVG(USPH_OC_AVG) OVER(ORDER BY DT_ID ASC
						ROWS BETWEEN 99 PRECEDING AND CURRENT ROW),
	total_return = usph_beta.total_return,
	cumulative_return = 100*usph_beta.cumm_return,
	annualized_return = 100*usph_beta.annual_return,
	volatility = usph_beta.historic_vaolatility,
	beta =  usph_beta.usph_beta,
	percentage_change = 100*((USPH_OC_AVG-prev_price)/prev_price)
FROM usph_beta
WHERE DT_ID IS NOT NULL;
GO


DECLARE @cadusd_buy DECIMAL(18,6);
SET @cadusd_buy = (SELECT buy_in_price FROM tick_table WHERE ticker = 'CADUSD');
DECLARE @cadusd_shares INTEGER;
SET @cadusd_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'CADUSD');
WITH count_cadusd AS (
	SELECT *, 
		LAG(CADUSD_OC_AVG) OVER(ORDER BY DT_ID) AS prev_price,
		COUNT(*) OVER(ORDER BY DT_ID ROWS BETWEEN 
			UNBOUNDED PRECEDING AND CURRENT ROW) AS row_num
	FROM cadusd_oc_avg
),
daily_log AS(
	SELECT *,
		LOG(CADUSD_OC_AVG/prev_price) AS log_d
	FROM count_cadusd
),
cadusd_daily_volatility AS(
	SELECT *,
		100*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
		BETWEEN 365 PRECEDING AND CURRENT ROW) AS daily_vaolatility
	FROM daily_log
),
cadusd_historic_vol AS(
	SELECT *,
	100*SQRT(365)*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
	BETWEEN 365 PRECEDING AND CURRENT ROW) AS historic_vaolatility
	FROM cadusd_daily_volatility
),
cadusd_total_returns AS(
	SELECT *,
		@cadusd_shares*(CADUSD_OC_AVG-@cadusd_buy) AS total_return
	FROM cadusd_historic_vol
),
cadusd_cummulative_returns AS(
	SELECT *,
		(CADUSD_OC_AVG-@cadusd_buy)/(@cadusd_buy) as cumm_return
	FROM cadusd_total_returns
),
cadusd_annualized_returns AS(--!!!!Change 252 to 365 for other non-stock/etf calculations
	SELECT *,
		annual_return = CASE
							WHEN cumm_return < -1.0 THEN 
									-1.0*POWER(ABS(cumm_return+1), 365.0/row_num)-1
							ELSE POWER(cumm_return+1, 365.0/row_num)-1
						END
	FROM cadusd_cummulative_returns
),
cadusd_daily_returns AS(
	SELECT acr.*,
		unified_date = CASE 
							WHEN market_idx.date_ID IS NULL THEN acr.DT_ID
							ELSE market_idx.date_ID
						END,
		100*((CADUSD_OC_AVG-prev_price)/(prev_price)) AS cadusd_daily_ret,
		market_idx.percentage_change AS mrkt_dailt_ret
		FROM cadusd_annualized_returns acr FULL JOIN (
		SELECT percentage_change, date_ID FROM portf_table
		WHERE ticker_ID = 'NDX') AS market_idx
		ON acr.DT_ID = market_idx.date_ID
),
cadusd_avg_daily_ret AS(
	SELECT *,
		AVG(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 365 PRECEDING AND CURRENT ROW) AS idx_avg_day_ret,
		AVG(cadusd_daily_ret) OVER(ORDER BY unified_date ROWS BETWEEN 365
				PRECEDING AND CURRENT ROW) AS cadusd_avg_day_ret
		FROM cadusd_daily_returns
),
cadusd_covar_sums AS(
		SELECT *,
			SUM(((cadusd_daily_ret)-(cadusd_avg_day_ret))*((mrkt_dailt_ret)-(idx_avg_day_ret)))
			OVER(ORDER BY unified_date ROWS BETWEEN 365 PRECEDING AND CURRENT ROW)--convert annual return from decimal to percent
			AS covar_sum
		FROM cadusd_avg_daily_ret
),
cadusd_variances AS(
		SELECT *,
			VAR(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 365 PRECEDING AND CURRENT ROW) AS var_idx,--252 or unbounded?
			covar_sum/(row_num-1) AS covar_stocks
		FROM cadusd_covar_sums
),
cadusd_beta AS(
		SELECT *,
		(covar_stocks/var_idx) AS cadusd_beta
		FROM cadusd_variances

)
INSERT INTO portf_table(date_ID, ticker_ID, day_10_avg, day_100_avg, total_return, cumulative_return,
		annualized_return, volatility, beta, percentage_change)
SELECT date_ID = DT_ID,
	ticker_ID = 'CADUSD',
	day_10_avg = AVG(CADUSD_OC_AVG) OVER(ORDER BY DT_ID ASC
							ROWS BETWEEN 9 PRECEDING AND CURRENT ROW),
	day_100_avg = AVG(CADUSD_OC_AVG) OVER(ORDER BY DT_ID ASC
						ROWS BETWEEN 99 PRECEDING AND CURRENT ROW),
	total_return = cadusd_beta.total_return,
	cumulative_return = 100*cadusd_beta.cumm_return,
	annualized_return = 100*cadusd_beta.annual_return,
	volatility = cadusd_beta.historic_vaolatility,
	beta =  cadusd_beta.cadusd_beta,
	percentage_change = 100*((CADUSD_OC_AVG-prev_price)/prev_price)
FROM cadusd_beta
WHERE DT_ID IS NOT NULL;
GO

SELECT * FROM portf_table;

--Add in data for our table as a whole
ALTER TABLE portf_table
ADD portf_value DECIMAL(18,6);

DECLARE @amd_shares INTEGER;
DECLARE @ijh_shares INTEGER;
DECLARE @hsy_shares INTEGER;
DECLARE @pg_shares INTEGER;
DECLARE @usph_shares INTEGER;
DECLARE @cadusd_shares INTEGER;
DECLARE @cl_shares INTEGER;
SET @amd_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'AMD');
SET @ijh_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'IJH');
SET @hsy_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'HSY');
SET @pg_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'PG');
SET @cl_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'CL');
SET @usph_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'USPH');
SET @cadusd_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'CADUSD');
UPDATE portf_table
SET portf_value = sum_val
FROM
(SELECT ((IJH_OC_AVG*@ijh_shares)+(AMD_OC_AVG*@amd_shares)+(CL_OC_AVG*@cl_shares)
					+(HSY_OC_AVG*@hsy_shares)+(PG_OC_AVG*@pg_shares)+
					(USPH_OC_AVG*@usph_shares)+(CADUSD_OC_AVG*@cadusd_shares)) as sum_val, 
Start_DT
FROM (SELECT * FROM portf_table pt INNER JOIN
		portfolio_OC_data pod ON FORMAT(pod.Start_DT, 'yyyy-MM-dd') =
									FORMAT(pt.date_ID, 'yyyy-MM-dd'))AS join_tbls
) AS sum_oc
WHERE FORMAT(Start_DT, 'yyyy-MM-dd') = FORMAT(portf_table.date_ID, 'yyyy-MM-dd');

SELECT (IJH_OC_AVG+AMD_OC_AVG+CL_OC_AVG+HSY_OC_AVG+PG_OC_AVG+USPH_OC_AVG+CADUSD_OC_AVG) as sum_val, 
Start_DT
FROM (SELECT * FROM portf_table pt INNER JOIN
		portfolio_OC_data pod ON FORMAT(pod.Start_DT, 'yyyy-MM-dd') =
									FORMAT(pt.date_ID, 'yyyy-MM-dd'))AS join_tbls;
GO


/*To do List:
	-get total, cumulative, annualized returns for portfolio(Done, although no divs included... fix later?)
	-get volatility of portfolio(Done)
	-Have sharpe ration for portfolio, NEED TO FILL IN THOUGH

	-Have total and cummulative returns per stock/etf(DONE)
	-have volatitlity(DONE)
	-10 and 100 day moving avgs(DONE)
	-Sharpe ratio?-this is kind of all I need still NEED
	-Have beta(DONE)

	-Price index? Just the price I guess? NEED
	-I think I have volatitlity of index(like NDX?)(DONE)
	-Comparison to total market index?(DONE-ish, do this in visualizations)

	-Have total and cumulative return(DONE) 
	-Have percent change(DONE)


Plan of action:
	-Figure out what is meant by the indicies(DONE)
	-Figure out sharpe ratio for individual stocks/etf
	-calculate stuff for portfolio(easy)
		-Gonna add sepearate ticker for portfolio, then just do what I did for all everything else
*/

--do what we did for individual investments for the portfolio
SELECT * FROM portf_table;
SELECT * FROM tick_table;
INSERT INTO tick_table(ticker, buy_in_price, full_name, number_purchased, portf_weight, div_value, div_date)
VALUES ('PORTF', (100000.0-0.378015414), 'Portfolio', 1, 1, 0.0, NULL);
SELECT * FROM portfolio_OC_data;

DECLARE @amd_shares INTEGER;
DECLARE @ijh_shares INTEGER;
DECLARE @hsy_shares INTEGER;
DECLARE @pg_shares INTEGER;
DECLARE @usph_shares INTEGER;
DECLARE @cadusd_shares INTEGER;
DECLARE @cl_shares INTEGER;
SET @amd_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'AMD');
SET @ijh_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'IJH');
SET @hsy_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'HSY');
SET @pg_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'PG');
SET @cl_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'CL');
SET @usph_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'USPH');
SET @cadusd_shares = (SELECT number_purchased FROM tick_table WHERE ticker = 'CADUSD');
DECLARE @portf_buy DECIMAL(18,6);
SET @portf_buy = (SELECT buy_in_price FROM tick_table WHERE ticker = 'PORTF');
WITH portf_creation AS(
	SELECT FORMAT(Start_DT, 'yyyy-MM-dd') AS DT_ID, PORTF_OC_AVG FROM(
		SELECT *,
			((IJH_OC_AVG*@ijh_shares)+(AMD_OC_AVG*@amd_shares)+(CL_OC_AVG*@cl_shares)
					+(HSY_OC_AVG*@hsy_shares)+(PG_OC_AVG*@pg_shares)+
					(USPH_OC_AVG*@usph_shares)+(CADUSD_OC_AVG*@cadusd_shares)) AS PORTF_OC_AVG
		FROM portfolio_OC_data
	) AS portf_data
),
count_portf AS (
	SELECT *, 
		LAG(PORTF_OC_AVG) OVER(ORDER BY DT_ID) AS prev_price,
		COUNT(*) OVER(ORDER BY DT_ID ROWS BETWEEN 
			UNBOUNDED PRECEDING AND CURRENT ROW) AS row_num
	FROM portf_creation
	WHERE PORTF_OC_AVG IS NOT NULL
),
daily_log AS(
	SELECT *,
		LOG(PORTF_OC_AVG/prev_price) AS log_d
	FROM count_portf
),
portf_daily_volatility AS(
	SELECT *,
		100*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
		BETWEEN 252 PRECEDING AND CURRENT ROW) AS daily_vaolatility
	FROM daily_log
),
portf_historic_vol AS(
	SELECT *,
	100*SQRT(252)*STDEV(log_d) OVER(ORDER BY DT_ID ROWS 
	BETWEEN 252 PRECEDING AND CURRENT ROW) AS historic_vaolatility
	FROM portf_daily_volatility
),
portf_total_returns AS(
	SELECT *,
		(PORTF_OC_AVG-@portf_buy) AS total_return
	FROM portf_historic_vol
),
portf_cummulative_returns AS(
	SELECT *,
		(total_return)/(@portf_buy) as cumm_return
	FROM portf_total_returns
),
portf_annualized_returns AS(--!!!!Change 252 to 365 for other non-stock/etf calculations
	SELECT *,
		annual_return = CASE
							WHEN cumm_return < -1.0 THEN 
									-1.0*POWER(ABS(cumm_return+1), 252.0/row_num)-1
							ELSE POWER(cumm_return+1, 252.0/row_num)-1
						END
	FROM portf_cummulative_returns
),
portf_daily_returns AS(
	SELECT acr.*,
		unified_date = CASE 
							WHEN market_idx.date_ID IS NULL THEN acr.DT_ID
							ELSE market_idx.date_ID
						END,
		100*((PORTF_OC_AVG-prev_price)/(prev_price)) AS portf_daily_ret,
		market_idx.percentage_change AS mrkt_dailt_ret
		FROM portf_annualized_returns acr FULL JOIN (
		SELECT percentage_change, date_ID FROM portf_table
		WHERE ticker_ID = 'NDX') AS market_idx
		ON acr.DT_ID = market_idx.date_ID
),
portf_avg_daily_ret AS(
	SELECT *,
		AVG(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 252 PRECEDING AND CURRENT ROW) AS idx_avg_day_ret,
		AVG(portf_daily_ret) OVER(ORDER BY unified_date ROWS BETWEEN 365
				PRECEDING AND CURRENT ROW) AS portf_avg_day_ret
		FROM portf_daily_returns
),
portf_covar_sums AS(
		SELECT *,
			SUM(((portf_daily_ret)-(portf_avg_day_ret))*((mrkt_dailt_ret)-(idx_avg_day_ret)))
			OVER(ORDER BY unified_date ROWS BETWEEN 365 PRECEDING AND CURRENT ROW)--convert annual return from decimal to percent
			AS covar_sum
		FROM portf_avg_daily_ret
),
portf_variances AS(
		SELECT *,
			VAR(mrkt_dailt_ret) OVER(ORDER BY unified_date
				ROWS BETWEEN 365 PRECEDING AND CURRENT ROW) AS var_idx,--252 or unbounded?
			covar_sum/(row_num-1) AS covar_stocks
		FROM portf_covar_sums
),
portf_beta AS(
		SELECT *,
		(covar_stocks/var_idx) AS portf_beta
		FROM portf_variances

)
INSERT INTO portf_table(date_ID, ticker_ID, day_10_avg, day_100_avg, total_return, cumulative_return,
annualized_return, volatility, beta, percentage_change)
SELECT date_ID = DT_ID,
	ticker_ID = 'PORTF',
	day_10_avg = AVG(PORTF_OC_AVG) OVER(ORDER BY DT_ID ASC
							ROWS BETWEEN 9 PRECEDING AND CURRENT ROW),
	day_100_avg = AVG(PORTF_OC_AVG) OVER(ORDER BY DT_ID ASC
						ROWS BETWEEN 99 PRECEDING AND CURRENT ROW),
	total_return = portf_beta.total_return,
	cumulative_return = 100*portf_beta.cumm_return,
	annualized_return = 100*portf_beta.annual_return,
	volatility = portf_beta.historic_vaolatility,
	beta =  portf_beta.portf_beta,
	percentage_change = 100*((PORTF_OC_AVG-prev_price)/prev_price)
FROM portf_beta
WHERE DT_ID IS NOT NULL;
GO

--reset sharpe, messed up calculation earlier
UPDATE portf_table
SET sharpe_ratio = NULL
GO

SELECT * FROM portf_table;


--Update stock/etf sharpe, and portf sharpe
UPDATE portf_table
SET sharpe_ratio = shrp_r FROM(
	SELECT date_ID,
		(exp_ret-3)/STDEV(exp_ret-3) OVER(ORDER BY date_ID ROWS BETWEEN 
										UNBOUNDED PRECEDING AND CURRENT ROW) AS shrp_r
	FROM (SELECT stock_data.date_ID, 
				3+beta*(exp_m_r - 3) AS exp_ret
		FROM (SELECT date_ID, beta FROM portf_table WHERE ticker_ID = 'AMD') AS stock_data
				INNER JOIN
			((SELECT date_ID, 
					AVG(cumulative_return) OVER(ORDER BY date_ID ROWS BETWEEN 
											UNBOUNDED PRECEDING AND CURRENT ROW)AS exp_m_r
			 FROM (SELECT date_ID, cumulative_return 
					FROM portf_table WHERE ticker_ID = 'NDX') AS ndx_data)) AS market_data
		ON stock_data.date_ID = market_data.date_ID) AS ret_data
) AS amd_shrp_calc
WHERE portf_table.ticker_ID = 'AMD' AND (portf_table.date_ID = amd_shrp_calc.date_ID);

UPDATE portf_table
SET sharpe_ratio = shrp_r FROM(
	SELECT date_ID,
		(exp_ret-3)/STDEV(exp_ret-3) OVER(ORDER BY date_ID ROWS BETWEEN 
											UNBOUNDED PRECEDING AND CURRENT ROW) AS shrp_r
	FROM (SELECT stock_data.date_ID, 
				3+beta*(exp_m_r - 3) AS exp_ret
		FROM (SELECT date_ID, beta FROM portf_table WHERE ticker_ID = 'CL') AS stock_data
				INNER JOIN
			((SELECT date_ID, 
					AVG(cumulative_return) OVER(ORDER BY date_ID ROWS BETWEEN 
											UNBOUNDED PRECEDING AND CURRENT ROW)AS exp_m_r
			 FROM (SELECT date_ID, cumulative_return 
					FROM portf_table WHERE ticker_ID = 'NDX') AS ndx_data)) AS market_data
		ON stock_data.date_ID = market_data.date_ID) AS ret_data
) AS cl_shrp_calc
WHERE portf_table.ticker_ID = 'CL' AND (portf_table.date_ID = cl_shrp_calc.date_ID);

UPDATE portf_table
SET sharpe_ratio = shrp_r FROM(
	SELECT date_ID,
		(exp_ret-3)/STDEV(exp_ret-3) OVER(ORDER BY date_ID ROWS BETWEEN 
											UNBOUNDED PRECEDING AND CURRENT ROW) AS shrp_r
	FROM (SELECT stock_data.date_ID, 
				3+beta*(exp_m_r - 3) AS exp_ret
		FROM (SELECT date_ID, beta FROM portf_table WHERE ticker_ID = 'HSY') AS stock_data
				INNER JOIN
			((SELECT date_ID, 
					AVG(cumulative_return) OVER(ORDER BY date_ID ROWS BETWEEN 
											UNBOUNDED PRECEDING AND CURRENT ROW)AS exp_m_r
			 FROM (SELECT date_ID, cumulative_return 
					FROM portf_table WHERE ticker_ID = 'NDX') AS ndx_data)) AS market_data
		ON stock_data.date_ID = market_data.date_ID) AS ret_data
) AS hsy_shrp_calc
WHERE portf_table.ticker_ID = 'HSY' AND (portf_table.date_ID = hsy_shrp_calc.date_ID);

UPDATE portf_table
SET sharpe_ratio = shrp_r FROM(
	SELECT date_ID,
		(exp_ret-3)/STDEV(exp_ret-3) OVER(ORDER BY date_ID ROWS BETWEEN 
											UNBOUNDED PRECEDING AND CURRENT ROW) AS shrp_r
	FROM (SELECT stock_data.date_ID, 
				3+beta*(exp_m_r - 3) AS exp_ret
		FROM (SELECT date_ID, beta FROM portf_table WHERE ticker_ID = 'IJH') AS stock_data
				INNER JOIN
			((SELECT date_ID, 
					AVG(cumulative_return) OVER(ORDER BY date_ID ROWS BETWEEN 
											UNBOUNDED PRECEDING AND CURRENT ROW)AS exp_m_r
			 FROM (SELECT date_ID, cumulative_return 
					FROM portf_table WHERE ticker_ID = 'NDX') AS ndx_data)) AS market_data
		ON stock_data.date_ID = market_data.date_ID) AS ret_data
) AS ijh_shrp_calc
WHERE portf_table.ticker_ID = 'IJH' AND (portf_table.date_ID = ijh_shrp_calc.date_ID);

UPDATE portf_table
SET sharpe_ratio = shrp_r FROM(
	SELECT date_ID,
		(exp_ret-3)/STDEV(exp_ret-3) OVER(ORDER BY date_ID ROWS BETWEEN 
											UNBOUNDED PRECEDING AND CURRENT ROW) AS shrp_r
	FROM (SELECT stock_data.date_ID, 
				3+beta*(exp_m_r - 3) AS exp_ret
		FROM (SELECT date_ID, beta FROM portf_table WHERE ticker_ID = 'PG') AS stock_data
				INNER JOIN
			((SELECT date_ID, 
					AVG(cumulative_return) OVER(ORDER BY date_ID ROWS BETWEEN 
											UNBOUNDED PRECEDING AND CURRENT ROW)AS exp_m_r
			 FROM (SELECT date_ID, cumulative_return 
					FROM portf_table WHERE ticker_ID = 'NDX') AS ndx_data)) AS market_data
		ON stock_data.date_ID = market_data.date_ID) AS ret_data
) AS pg_shrp_calc
WHERE portf_table.ticker_ID = 'PG' AND (portf_table.date_ID = pg_shrp_calc.date_ID);


UPDATE portf_table
SET sharpe_ratio = shrp_r FROM(
	SELECT date_ID,
		(exp_ret-3)/STDEV(exp_ret-3) OVER(ORDER BY date_ID ROWS BETWEEN 
											UNBOUNDED PRECEDING AND CURRENT ROW) AS shrp_r
	FROM (SELECT stock_data.date_ID, 
				3+beta*(exp_m_r - 3) AS exp_ret
		FROM (SELECT date_ID, beta FROM portf_table WHERE ticker_ID = 'USPH') AS stock_data
				INNER JOIN
			((SELECT date_ID, 
					AVG(cumulative_return) OVER(ORDER BY date_ID ROWS BETWEEN 
											UNBOUNDED PRECEDING AND CURRENT ROW)AS exp_m_r
			 FROM (SELECT date_ID, cumulative_return 
					FROM portf_table WHERE ticker_ID = 'NDX') AS ndx_data)) AS market_data
		ON stock_data.date_ID = market_data.date_ID) AS ret_data
) AS usph_shrp_calc
WHERE portf_table.ticker_ID = 'USPH' AND (portf_table.date_ID = usph_shrp_calc.date_ID);

--Shapre Calculation will be a little different than individual stocks
DECLARE @amd_w FLOAT;
DECLARE @ijh_w FLOAT;
DECLARE @hsy_w FLOAT;
DECLARE @pg_w FLOAT;
DECLARE @usph_w FLOAT;
DECLARE @cadusd_w FLOAT;
DECLARE @cl_w FLOAT;
SET @amd_w = (SELECT portf_weight FROM tick_table WHERE ticker = 'AMD');
SET @ijh_w = (SELECT portf_weight FROM tick_table WHERE ticker = 'IJH');
SET @hsy_w = (SELECT portf_weight FROM tick_table WHERE ticker = 'HSY');
SET @pg_w = (SELECT portf_weight FROM tick_table WHERE ticker = 'PG');
SET @cl_w = (SELECT portf_weight FROM tick_table WHERE ticker = 'CL');
SET @usph_w = (SELECT portf_weight FROM tick_table WHERE ticker = 'USPH');
SET @cadusd_w = (SELECT portf_weight FROM tick_table WHERE ticker = 'CADUSD');
WITH amd_exp_r AS(
	SELECT stock_data.date_ID, 
			3+beta*(exp_m_r - 3) AS exp_ret
	FROM (SELECT date_ID, beta FROM portf_table WHERE ticker_ID = 'AMD') AS stock_data
			INNER JOIN
		((SELECT date_ID, 
				AVG(cumulative_return) OVER(ORDER BY date_ID ROWS BETWEEN 
										UNBOUNDED PRECEDING AND CURRENT ROW)AS exp_m_r
		FROM (SELECT date_ID, cumulative_return 
			  FROM portf_table WHERE ticker_ID = 'NDX') AS ndx_data)) AS market_data
	ON stock_data.date_ID = market_data.date_ID
),
cadusd_exp_r AS(
	SELECT stock_data.date_ID, 
			3+beta*(exp_m_r - 3) AS exp_ret
	FROM (SELECT date_ID, beta FROM portf_table WHERE ticker_ID = 'CADUSD') AS stock_data
			INNER JOIN
		((SELECT date_ID, 
				AVG(cumulative_return) OVER(ORDER BY date_ID ROWS BETWEEN 
											UNBOUNDED PRECEDING AND CURRENT ROW)AS exp_m_r
		FROM (SELECT date_ID, cumulative_return 
				FROM portf_table WHERE ticker_ID = 'NDX') AS ndx_data)) AS market_data
	ON stock_data.date_ID = market_data.date_ID
),
cl_exp_r AS(
	SELECT stock_data.date_ID, 
			3+beta*(exp_m_r - 3) AS exp_ret
	FROM (SELECT date_ID, beta FROM portf_table WHERE ticker_ID = 'CL') AS stock_data
			INNER JOIN
		((SELECT date_ID, 
				AVG(cumulative_return) OVER(ORDER BY date_ID ROWS BETWEEN 
										UNBOUNDED PRECEDING AND CURRENT ROW)AS exp_m_r
		FROM (SELECT date_ID, cumulative_return 
				FROM portf_table WHERE ticker_ID = 'NDX') AS ndx_data)) AS market_data
	ON stock_data.date_ID = market_data.date_ID
),
hsy_exp_r AS(
	SELECT stock_data.date_ID, 
			3+beta*(exp_m_r - 3) AS exp_ret
	FROM (SELECT date_ID, beta FROM portf_table WHERE ticker_ID = 'HSY') AS stock_data
			INNER JOIN
		((SELECT date_ID, 
				AVG(cumulative_return) OVER(ORDER BY date_ID ROWS BETWEEN 
										UNBOUNDED PRECEDING AND CURRENT ROW)AS exp_m_r
		FROM (SELECT date_ID, cumulative_return 
				FROM portf_table WHERE ticker_ID = 'NDX') AS ndx_data)) AS market_data
	ON stock_data.date_ID = market_data.date_ID
),
ijh_exp_r AS(
	SELECT stock_data.date_ID, 
			3+beta*(exp_m_r - 3) AS exp_ret
	FROM (SELECT date_ID, beta FROM portf_table WHERE ticker_ID = 'IJH') AS stock_data
			INNER JOIN
		((SELECT date_ID, 
				AVG(cumulative_return) OVER(ORDER BY date_ID ROWS BETWEEN 
										UNBOUNDED PRECEDING AND CURRENT ROW)AS exp_m_r
		FROM (SELECT date_ID, cumulative_return 
				FROM portf_table WHERE ticker_ID = 'NDX') AS ndx_data)) AS market_data
	ON stock_data.date_ID = market_data.date_ID
),
pg_exp_r AS(
	SELECT stock_data.date_ID, 
			3+beta*(exp_m_r - 3) AS exp_ret
	FROM (SELECT date_ID, beta FROM portf_table WHERE ticker_ID = 'PG') AS stock_data
			INNER JOIN
		((SELECT date_ID, 
				AVG(cumulative_return) OVER(ORDER BY date_ID ROWS BETWEEN 
										UNBOUNDED PRECEDING AND CURRENT ROW)AS exp_m_r
		FROM (SELECT date_ID, cumulative_return 
				FROM portf_table WHERE ticker_ID = 'NDX') AS ndx_data)) AS market_data
	ON stock_data.date_ID = market_data.date_ID
),
usph_exp_r AS(
	SELECT stock_data.date_ID, 
			3+beta*(exp_m_r - 3) AS exp_ret
	FROM (SELECT date_ID, beta FROM portf_table WHERE ticker_ID = 'USPH') AS stock_data
			INNER JOIN
		((SELECT date_ID, 
				AVG(cumulative_return) OVER(ORDER BY date_ID ROWS BETWEEN 
										UNBOUNDED PRECEDING AND CURRENT ROW)AS exp_m_r
		FROM (SELECT date_ID, cumulative_return 
				FROM portf_table WHERE ticker_ID = 'NDX') AS ndx_data)) AS market_data
	ON stock_data.date_ID = market_data.date_ID
),
investments_exp_r AS(
	SELECT acchip.*,
		uer.exp_ret AS usph_exp_ret
	FROM usph_exp_r uer FULL JOIN(
		SELECT acchi.*,
			per.exp_ret AS pg_exp_ret
		FROM pg_exp_r per FULL JOIN(
			SELECT acch.*,
				ier.exp_ret AS ijh_exp_ret
			FROM ijh_exp_r ier FULL JOIN(
				SELECT acc.*,
					her.exp_ret AS hsy_exp_ret
				FROM hsy_exp_r her FULL JOIN(
					SELECT aca.*,
						cler.exp_ret AS cl_exp_ret
					FROM cl_exp_r cler FULL JOIN(
						SELECT aer.exp_ret AS amd_exp_ret,
							caer.exp_ret AS cadusd_exp_ret,
							caer.date_ID
						FROM amd_exp_r aer FULL JOIN cadusd_exp_r caer
						ON aer.date_ID = caer.date_ID
					) AS aca
					ON cler.date_ID = aca.date_ID
				)AS acc
				ON acc.date_ID = her.date_ID
			)AS acch
			ON acch.date_ID = ier.date_ID
		)AS acchi
		ON acchi.date_ID = per.date_ID
	)AS acchip
	ON acchip.date_ID = uer.date_ID

),
portf_exp_r AS(
	SELECT date_ID,
		((@amd_w*amd_exp_ret)+(@cadusd_w*cadusd_exp_ret)+(@cl_w*cl_exp_ret)+(@hsy_w*hsy_exp_ret)+
		(@ijh_w*ijh_exp_ret)+(@pg_w*pg_exp_ret)+(@usph_w*usph_exp_ret)) AS portf_exp_ret
	FROM investments_exp_r
),
portf_sharpe AS(
	SELECT date_ID,
		(portf_exp_ret-3)/STDEV(portf_exp_ret-3) OVER(ORDER BY date_ID ROWS BETWEEN UNBOUNDED
															PRECEDING AND CURRENT ROW) AS sharpe_val
	FROM portf_exp_r
)
UPDATE portf_table
SET sharpe_ratio = sharpe_val
FROM portf_sharpe
WHERE portf_table.ticker_ID = 'PORTF' AND (portf_table.date_ID = portf_sharpe.date_ID);
GO

SELECT * FROM portf_table;